# Time Dilation by Example
Let's apply our knowledge of Special Relativity (SR) and General Relativity (GR) to predict the phenomenon of time dilation.

Special Relativity, which describes spacetime far from a significant gravity field, explains how a stationary observer perceives time passing slower for a moving observer when the relative velocity between them is very large.

General Relativity, which deals with gravity, explains how time slows down in proximity to a large gravitational acceleration.

The [wikipedia page for time dilation](https://en.wikipedia.org/wiki/Time_dilation) provides a derivation of these effects and a handy plot showing time dilation in Earth orbit. Let's try to recreate it.

![time dilation plot](./Time_Dilation_vs_Orbital_Height.png)

We'll use Python because it's quick and perfect for playing around like this.

For math and vectors, we'll use good ol' numpy.

For universal constants and tracking high-precision quantities, I suggest using [Nyx](https://nyxspace.com/nyxspace/showcase/). It can do a lot more than we're using it for and it is good enough to plan a trajectory around the Moon, so it should be good enough for us.

Nyx depends on another package, [Hifitime](https://nyxspace.com/hifitime/intro/), which is designed to keep time with nanosecond precision across a 65k year span for exactly this type of use case. Hifitime is available through Nyx via the `time` module.

In [1]:
import numpy as np
import nyx_space as nyx

GRAV_CONST = nyx.? # gravitational constant
SPEED_OF_LIGHT = nyx.? # speed of light

## The Schwartzschild solution

Let's see how far we can get by crunching the numbers on the [Schwartzschild solution](https://en.wikipedia.org/wiki/Schwarzschild_solution) to the Einstein field equations as they appear on Wikipedia.

$$
d{t_E}^2 = \Big(1 + \frac{2 U}{c^2} \Big) d{t_c}^2 - \Big(1 + \frac{2 U}{c^2} \Big)^{-1} v^2, \quad U = \frac{-GM_i}{r_i}, \quad v^2 = \frac{dx^2 + dy^2 + dz^2}{c^2}
$$

where:
* $dt_E$ is a small increment of proper time $t_E$ (an interval that could be recorded on an atomic clock),
* $dt_c$ is a small increment in the coordinate $t_c$ (coordinate time),
* $dx$, $dy$, $dz$, are small increments in the three coordinates $x$, $y$, $z$ of the clock's position,
* $\frac{-GM_i}{r_i}$ represents the sum of the Newtonian gravitational potentials due to the masses in the neighborhood, based on their distances $r_i$ from the clock. This sum includes any tidal potentials.

Next we rearrange the formula to the form we want to plot: proper time over coordinate time with respect to orbital altitude at a given instant.

$$
\frac{dt_E}{dt_c} = \sqrt{1 + \frac{2U}{c^2} - \frac{v^2}{c^2} + \Big(\frac{c^2}{2U} + 1 \Big)^{-1} \frac{{v_{||}^2}}{c^2}}
$$

where:
* $v_{||}$ is the radial velocity,
* $U = \frac{-GM_i}{r_i}$ is the Newtownian potential,


Now add some more sugar to the syntax here to shape the equation into something a little more manageable.

let:
* $v_e = \sqrt{\frac{2GM_i}{r_i}}$ be the escape speed; hence ${v_e}^2 = -2U$
* $\beta = v/c$, $\beta_e = v_e/c$, and $\beta_{||} = v_e/c$ be velocities as a percentage of the speed of light $c$,

$$
\frac{dt_E}{dt_c} = \sqrt{1 - \Big( \beta^2 + {\beta_e}^2 + \frac{ {\beta_{||}}^2 {\beta_e}^2 }{ 1 - {\beta_e}^2 } \Big) }
$$

## Assumptions & Simplifications
We can make a few simplifications to the full equation in order to recreate the plot from Wikipedia.

* All orbits under consideration are circular.
* All orbits are computed at the same $x$ and $y$ coordinates.
* Mean parameters are used for Earth's radius and mass.
* The only significant Newtonian gravitational potentials are from Earth, Moon, and Sun.

In [ ]:
def newtonian_potential(mass, radius):
    """
        U_i = -G M_i / r_i
    """
    return -GRAV_CONST * mass / radius

def escape_speed(total_potential):
    """
        v_e = sqrt(-2 * U)
    """
    return np.sqrt(-2 * total_potential)

def percent_c(velocity):
    """
        Beta = v / c
    """
    return velocity / SPEED_OF_LIGHT

In [ ]:
def time_dilation_circular_orbit(orbit_altitude):
    pass